In [74]:
!pip install "langchain>=0.2.16" "langchain-aws>=0.2.2" boto3 requests beautifulsoup4 psycopg2-binary tqdm sentence-transformers langchain_community pyvi


In [75]:
from dotenv import load_dotenv
# bedrock_pgvector_demo.py
import os, re, json, hashlib, math
import requests
from bs4 import BeautifulSoup
from datetime import datetime

load_dotenv()
from typing import List, Tuple
from tqdm import tqdm

import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_aws import BedrockEmbeddings

import psycopg2
from psycopg2.extras import execute_values

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings  # local CPU/GPU
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

WIKI_URL = "https://vi.wikipedia.org/wiki/Chi%E1%BA%BFn_tranh_Vi%E1%BB%87t_Nam"
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DATABASE")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
REGION = os.getenv("AWS_REGION")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
# MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
# MODEL_NAME = "dangvantuan/vietnamese-embedding"
MODEL_NAME = "AITeamVN/Vietnamese_Embedding"
PG_DSN   = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

In [76]:
# -------- Utils: tải & làm sạch wiki --------
def fetch_wiki(url: str) -> Tuple[str, str, str]:
    r = requests.get(url, headers={"User-Agent": "RAG-Pgvector-Local"}, timeout=30)
    r.raise_for_status()
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(r.text, "html.parser")
    title = soup.find("h1", id="firstHeading")
    title = title.get_text(strip=True) if title else (soup.title.get_text(strip=True) if soup.title else url)
    lang = soup.find("html").get("lang") if soup.find("html") else "vi"
    content_div = soup.find("div", id="mw-content-text")
    text = content_div.get_text(separator="\n", strip=True) if content_div else soup.get_text("\n", strip=True)
    text = re.sub(r"\[\d+\]", "", text)      # bỏ [1], [2]...
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{2,}", "\n\n", text).strip()
    return title, lang, text

def split_text(text: str) -> List[str]:
    """
    Hàm cũ: Chia text bằng RecursiveCharacterTextSplitter
    """
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=1000, chunk_overlap=150, length_function=len
    )
    chunks = [c.page_content for c in splitter.create_documents([text])]
    return [c for c in chunks if len(c) >= 50]

def split_text_by_paragraphs(text: str) -> List[str]:
    """
    Hàm mới: Chia text theo từng đoạn văn, đảm bảo mỗi chunk tối đa 500 từ
    """
    def count_words(text: str) -> int:
        """Đếm số từ trong text tiếng Việt sử dụng pyvi"""
        try:
            # tokenized = tokenize(text)
            words = text.split()
            return len(words)
        except:
            # Fallback nếu pyvi không hoạt động
            return len(text.split())
    
    # text_send = tokenize(text)
    text_send = text

    # Chia theo đoạn văn (double newline)
    paragraphs = text_send.split('.\n')

    # Lọc bỏ các đoạn quá ngắn và làm sạch
    chunks = []
    for para in paragraphs:
        para = para.strip()
        # remove /n
        para = para.replace('\n', ' ')
        word_count = count_words(para)
        
        # Chỉ giữ các đoạn có tối thiểu 10 từ
        if word_count < 10:
            continue
            
        # Nếu đoạn có độ dài hợp lý (10-500 từ), thêm trực tiếp
        if word_count <= 200:
            chunks.append(para)
        else:
            # Nếu đoạn quá dài (>500 từ), chia nhỏ hơn theo câu
            sentences = re.split(r'\.[\s]+', para)
            current_chunk = ""
            
            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue
                    
                # Thêm dấu chấm nếu câu chưa có
                if not sentence.endswith('.'):
                    sentence += '.'
                
                # Kiểm tra nếu thêm câu này vào chunk hiện tại có vượt quá 500 từ không
                test_chunk = current_chunk + " " + sentence if current_chunk else sentence
                test_word_count = count_words(test_chunk)
                
                if test_word_count <= 200:
                    current_chunk = test_chunk
                else:
                    # Lưu chunk hiện tại nếu đủ dài (tối thiểu 10 từ)
                    if count_words(current_chunk) >= 10:
                        chunks.append(current_chunk.strip())
                    current_chunk = sentence
            
            # Lưu chunk cuối cùng nếu đủ dài
            if count_words(current_chunk) >= 10:
                chunks.append(current_chunk.strip())
    
    return chunks

# -------- Embeddings (LOCAL) --------
def get_embedder():
    # HuggingFaceEmbeddings tải model local qua sentence-transformers
    return HuggingFaceEmbeddings(model_name=MODEL_NAME, model_kwargs={"use_auth_token": HUGGINGFACEHUB_API_TOKEN})

def detect_dims(embedder) -> int:
    v = embedder.embed_query("ping")
    return len(v)

def embed_query(embedder, query: str) -> List[float]:
    """Helper function to embed a single query"""
    return embedder.embed_query(query)

In [77]:
def db_connect():
    return psycopg2.connect(PG_DSN)

def upsert_document(conn, url: str, title: str, lang: str, checksum: bytes) -> int:
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO rag.document (source_url, title, language, checksum)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (source_url) DO UPDATE
              SET title = EXCLUDED.title, language = EXCLUDED.language, checksum = EXCLUDED.checksum
            RETURNING id
        """, (url, title, lang, psycopg2.Binary(checksum)))
        doc_id = cur.fetchone()[0]
        conn.commit()
        return doc_id

def insert_chunks(conn, doc_id: int, chunks: List[str], vectors: List[List[float]]):
    # Chèn bulk; pgvector literal là chuỗi '[v1,v2,...]'
    rows = []
    for i, (txt, vec) in enumerate(zip(chunks, vectors)):
        vec_str = "[" + ",".join(f"{x:.7f}" for x in vec) + "]"
        rows.append((
            doc_id, i, txt, None, len(txt), vec_str
        ))
    with conn.cursor() as cur:
        execute_values(cur, """
            INSERT INTO rag.chunk (document_id, ordinal, content, n_tokens, char_count, embedding)
            VALUES %s
        """, rows)
    conn.commit()

# ---------- 5) SEARCH ----------
def semantic_search(conn, query_vec: List[float], top_k=5):
    vec_str = "[" + ",".join(f"{x:.7f}" for x in query_vec) + "]"
    sql = """
        SELECT id, content,
               1 - (embedding <=> %s::vector) AS cosine_similarity
        FROM rag.chunk
        ORDER BY embedding <=> %s::vector ASC
        LIMIT %s
    """
    with conn.cursor() as cur:
        cur.execute(sql, (vec_str, vec_str, top_k))
        return cur.fetchall()


def hybrid_search(conn, query_text: str, query_vec: List[float], top_k=5, rrf_k=60):
    """
    Reciprocal Rank Fusion (RRF): f = 1/(K + rank)
    K ~ 60 là con số thực nghiệm phổ biến.
    """
    vec_str = "[" + ",".join(f"{x:.7f}" for x in query_vec) + "]"
    sql = f"""
    WITH sem AS (
      SELECT id, content,
             ROW_NUMBER() OVER (ORDER BY embedding <=> %s::vector ASC) AS rnk_sem
      FROM rag.chunk
      LIMIT 200
    ),
    fts AS (
      SELECT id, content,
             ROW_NUMBER() OVER (ORDER BY ts_rank_cd(content_tsv, plainto_tsquery('simple', %s)) DESC) AS rnk_fts
      FROM rag.chunk
      WHERE content_tsv @@ plainto_tsquery('simple', %s)
      LIMIT 200
    ),
    combo AS (
      SELECT COALESCE(sem.id, fts.id) AS id,
             COALESCE(sem.content, fts.content) AS content,
             (CASE WHEN sem.rnk_sem IS NULL THEN 0 ELSE 1.0/(%s + sem.rnk_sem) END) +
             (CASE WHEN fts.rnk_fts IS NULL THEN 0 ELSE 1.0/(%s + fts.rnk_fts) END) AS rrf
      FROM sem FULL OUTER JOIN fts USING (id)
    )
    SELECT id, content
    FROM combo
    ORDER BY rrf DESC
    LIMIT %s;
    """
    with conn.cursor() as cur:
        cur.execute(sql, (vec_str, query_text, query_text, rrf_k, rrf_k, top_k))
        return cur.fetchall()

In [78]:
print("== Tải Wikipedia ==")
title, lang, text = fetch_wiki(WIKI_URL)
checksum = hashlib.sha256(text.encode("utf-8")).digest()
print(f"Title: {title} | Lang: {lang} | Length: {len(text)} chars")

print("\n== So sánh 2 phương pháp chia chunks ==")

# Phương pháp cũ - RecursiveCharacterTextSplitter
chunks_old = split_text(text)
print(f"Chunks (phương pháp cũ - RecursiveCharacterTextSplitter): {len(chunks_old)}")
if chunks_old:
    print(f"Độ dài chunk đầu tiên: {len(chunks_old[0])} ký tự")
    print(f"Preview: {chunks_old[0][:200]}...")

print()

# Phương pháp mới - chia theo đoạn văn
chunks_new = split_text_by_paragraphs(text)
print(f"Chunks (phương pháp mới - theo đoạn văn): {len(chunks_new)}")
if chunks_new:
    print(f"Độ dài chunk đầu tiên: {len(chunks_new[0])} ký tự")
    print(f"Preview: {chunks_new[0][:200]}...")

# Sử dụng phương pháp mới cho các bước tiếp theo
chunks = chunks_new
print(f"\n=> Sử dụng phương pháp mới với {len(chunks)} chunks")

== Tải Wikipedia ==
Title: Chiến tranh Việt Nam | Lang: vi | Length: 364953 chars

== So sánh 2 phương pháp chia chunks ==
Chunks (phương pháp cũ - RecursiveCharacterTextSplitter): 458
Độ dài chunk đầu tiên: 971 ký tự
Preview: Bài này
quá dài
, khiến việc đọc và tìm kiếm thông tin trở nên khó khăn.
Bạn có thể
tách
nội dung thành các bài nhỏ hơn,
cô đọng
nội dung lại, và thêm bớt
các đề mục con
.
(
tháng 2/2024
)
Về lịch sử ...

Chunks (phương pháp mới - theo đoạn văn): 921
Độ dài chunk đầu tiên: 71 ký tự
Preview: Bài này quá dài , khiến việc đọc và tìm kiếm thông tin trở nên khó khăn...

=> Sử dụng phương pháp mới với 921 chunks


In [79]:
print("== Khởi tạo LOCAL embeddings ==")
embedder = get_embedder()
dims = detect_dims(embedder)   # <- tự nhận kích thước vector
print("Embedding dims:", dims)

print("== Tạo vector ==")
vectors = embedder.embed_documents(chunks)

conn = db_connect()
print("== Lưu document & chunks ==")
doc_id = upsert_document(conn, WIKI_URL, title, lang, checksum)
insert_chunks(conn, doc_id, chunks, vectors)

== Khởi tạo LOCAL embeddings ==


c:\Users\thanh\anaconda3\envs\python312\Lib\site-packages\sentence_transformers\SentenceTransformer.py:196: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


Embedding dims: 1024
== Tạo vector ==
== Lưu document & chunks ==


In [82]:
print("== Truy vấn thử (semantic) ==")
conn = db_connect()
q = "Mối liên hệ giữa chiến tranh Việt Nam và chiến tranh Đông Phương ?"
# q_next = tokenize(q)
# print("Query:", q_next)
qvec = embed_query(embedder, q)
rows = semantic_search(conn, qvec, top_k=10)
for rid, content, sim in rows:
    print(f"\n[#{rid}] cos_sim={sim:.4f}\n{content[:100]}...")

print("\n== Truy vấn thử (hybrid RRF) ==")
rows = hybrid_search(conn, q, qvec, top_k=10)
for rid, content in rows:
    print(f"\n[#{rid}] {content[:100]}...")

conn.close()

== Truy vấn thử (semantic) ==

[#387] cos_sim=0.4041
Sự cạnh tranh chiến tranh lạnh với Liên Xô là mối quan tâm lớn nhất về chính sách đối ngoại của Mỹ t...

[#993] cos_sim=0.3628
Among them are: encouragement to national aspirations under non-Communist leadership for peoples of ...

[#362] cos_sim=0.3529
Sau khi chiến tranh kết thúc, sự chia rẽ Trung-Xô xảy ra kết hợp mâu thuẫn giữa nhà nước Việt Nam th...

[#882] cos_sim=0.3439
[ 1 ] Các báo cáo của chính phủ Hoa Kỳ hiện cũng trích dẫn ngày 1 tháng 11 năm 1955 là thời điểm bắt...

[#440] cos_sim=0.2239
Đồng thời với việc từ chối tuyển cử, chế độ Việt Nam Cộng hòa ra sức củng cố quyền lực, đàn áp khốc ...

[#482] cos_sim=0.1975
Quốc hội Việt Nam Dân chủ Cộng hòa trong Nghị quyết ngày 20 tháng 9 năm 1955 tuyên bố: Dựa trên bản ...

[#364] cos_sim=0.1846
[ 71 ] Một nguồn khác thống kê rằng tổng lượng chất nổ mà quân đội Hoa Kỳ sử dụng trong chiến tranh ...

[#1256] cos_sim=0.1587
Summons of Trumpet: U.S.–Vietnam in Perspective . Novato,